In this post, I'd like to look at three stochastic dynamical systems, each a little more complex than the next.
We'll mainly be concerned with differential equations of the form

$$\dot z = f(z) + \sigma\dot\xi$$

where the trajectory $z$ that we're solving for lives in $\mathbb{R}^n$, $f : \mathbb{R}^n \to \mathbb{R}^n$ is the deterministic part of the dynamics, $\dot\xi$ is random *white noise*, and $\sigma$ is the noise amplitude.
In general, $\sigma$ could be a matrix that depends on the solution $z$ itself, but for the following we'll only consider cases where $\sigma$ is a scalar constant.

If you haven't seen stochastic differential equations before, you might be wondering at this point what I mean by "white noise".
White noise is characterized as a Gaussian stochastic process with zero mean and covariance function

$$\langle \dot\xi(s)\dot\xi(t)\rangle = \delta(t - s)$$

where $\delta$ is the Dirac delta function and $\langle\cdot\rangle$ denotes statistical expectation.
Like the delta function, white noise is a bit of a convenient fiction that we tell ourselves in order to get on with our lives -- there is no well-defined stochastic process with these properties.
The full details of how to define [stochastic integration](https://en.wikipedia.org/wiki/It%C3%B4_calculus) are... pretty rowdy and I don't want to go there.
If you do want to see the gory details, you can refer to the books by [Gardiner](https://link.springer.com/book/9783540707127) or [Øksendal](https://doi.org/10.1007/978-3-642-14394-6).

For our purposes, it's enough to know that the solutions of this stochastic differential equation can emerge through a limit of discrete-time processes

$$z_{n + 1} = z_n + \delta t\cdot F(z_n, z_{n + 1}) + \sqrt{\delta t}\sigma\dot\xi_n,$$

where $\dot\xi_n$ are uncorrelated mean-zero normal random variables with variance 1, and $F$ is some function of the current and past state that makes for a consistent discretization of the deterministic part of the ODE.
(I'm leaving room here for both explicit and implicit time discretizations.)
The first process that we'll look at can be solved in closed form, but the next two have no analytical solution and so numerical methods are our only recourse.

### The Ornstein-Uhlenbeck process

The simplest stochastic dynamical systems we can consider are the linear ones:

$$\dot z = -Az + \sigma\dot\xi,$$

where $A$ is a symmetric positive-definite matrix.
Ornstein originally used this system as a model for the velocity of a particle under the combined influence of friction and random collision with neighboring particles.
With the notation above, $A$ is the friction coefficient and $\sigma$ is the collision intensity.

Much like for deterministic ODE, we can use the matrix exponential and the usual [variation of parameters](https://en.wikipedia.org/wiki/Variation_of_parameters) trick to write down an explicit solution of this SDE:

$$z(t) = e^{-tA}z(0) + \sigma\int_0^te^{-(t - s)A}\dot\xi(s)ds.$$

Since the noise $\dot\xi$ is a Gaussian process and any finite sum of normal random variables is also normal, the solution $z$ is also a Gaussian process.
If we can compute the mean and covariance function, then we know almost everything we need about the process.

First, taking expectations can be interchanged with time integrals.
So if we take the expected value of the solution we'll be left with just the exponentially-decaying influence of the initial condition:

$$\mu(t) = \langle z(t)\rangle = e^{-tA}\langle z(0)\rangle.$$

The covariance function is much more interesting:

$$\begin{align}
\sigma^{-2}\langle (z(t_1) - \mu(t_1))(z(t_2) - \mu(t_2))^*\rangle & = \int_0^{t_1}\int_0^{t_2}e^{-(t_1 - s_1)A}e^{-(t_2 - s_2)A}\langle\dot\xi(s_1)\dot\xi(s_2)^*\rangle ds_1\,ds_2 \\
& = \int_0^{t_1}\int_0^{t_2}e^{-(t_1 - s_1)A}e^{-(t_2 - s_2)A}\delta(s_1 - s_2)ds_1\,ds_2
\end{align}$$